<a href="https://colab.research.google.com/github/Map987/download-anime-mag-/blob/main/downimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Map987/download-anime-mag-.git

Cloning into 'download-anime-mag-'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 79 (delta 8), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (79/79), 2.51 MiB | 2.87 MiB/s, done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

folders = ["/content/sample_data/outputtxt", "/content/sample_data/outtxt", "/content/sample_data/imagebig/"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

In [4]:
import os
import re
import xml.etree.ElementTree as ET
from typing import List

def extract_links(xml_string: str) -> List[str]:
    root = ET.fromstring(xml_string)
    links = []
    for url in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
        image_loc = url.find('{http://www.google.com/schemas/sitemap-image/1.1}image').find('{http://www.google.com/schemas/sitemap-image/1.1}loc').text
        if image_loc:
            links.append(image_loc)
    return links

In [5]:
def write_links_to_file(file_path: str, links: List[str]) -> None:
    with open(file_path, 'w') as f:
        for link in links:
            f.write(link)

In [6]:
input_folder = '/content/download-anime-mag-/'
output_folder = '/content/sample_data/outputtxt/'

for file_name in os.listdir(input_folder):
    if file_name.endswith('.xml'):
        input_path = os.path.join(input_folder, file_name)
        with open(input_path, 'r') as f:
            xml_string = f.read()
        links = extract_links(xml_string)
        output_path = os.path.join(output_folder, file_name[:-4] + '.txt')
        write_links_to_file(output_path, links)
        print(f'{output_path} written successfully.\n')

/content/sample_data/outputtxt/image-sitemap-44.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-15.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-53.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-48.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-9.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-10.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-30.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-43.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-23.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-38.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-52.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-13.txt written successfully.

/content/sample_data/outputtxt/image-sitemap-57_1.txt written successfully.

/content/sample_data/out

In [7]:
import os

input_folder_path = '/content/sample_data/outputtxt/'
output_folder_path = '/content/sample_data/outtxt/'

# get all .txt files in input folder
files = [f for f in os.listdir(input_folder_path) if os.path.isfile(os.path.join(input_folder_path, f)) and f.endswith('.txt')]

for file_name in files:
    # open input file
    input_file_path = os.path.join(input_folder_path, file_name)
    with open(input_file_path, 'r') as input_file:
        file_content = input_file.read()

    # add newline before each https
    file_content = file_content.replace('https://', 'https://')

    # create output file
    output_file_path = os.path.join(output_folder_path, file_name)
    with open(output_file_path, 'w') as output_file:
        output_file.write(file_content)

In [8]:
import os

outtxt_dir = "/content/sample_data/outtxt"
data_dir = "/content/sample_data/"

# 遍历outtxt_dir目录中的所有txt文件
for filename in os.listdir(outtxt_dir):
    if filename.endswith(".txt") and filename.startswith("image-sitemap-"):
        # 获取X值
        x = filename.split("-")[-1].split(".")[0]
        
        # 创建imageX文件夹
        folder = os.path.join(data_dir, f"image{x}")
        os.makedirs(folder, exist_ok=True)

In [15]:
import requests
import os
import zipfile
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def download(url, output_folder_path):
    response = requests.get(url)
    file_name = os.path.basename(url)
    file_path = os.path.join(output_folder_path, file_name)

    with open(file_path, 'wb') as file:
        file.write(response.content)
        print(f"File {file_name} downloaded successfully.")

def process_txt(x):
    url_file_path = f"/content/sample_data/outtxt/image-sitemap-{x}.txt"
    output_folder_path = f"/content/sample_data/image{x}/"
    destination_folder = '/content/sample_data/imagebig'

    try:
        with open(url_file_path, 'r') as url_file:
            file_content = url_file.read()

            urls = file_content.split('https')
            urls = ['https' + url for url in urls if url.endswith('.jpg') or url.endswith('.png')]

            count = len(urls)
            with ThreadPoolExecutor(max_workers=64) as executor:
                futures = [executor.submit(download, url, output_folder_path) for url in urls]
                for future in as_completed(futures):
                    future.result()

        print(f"Downloaded {count} files for image{x}.")

        source_folder = output_folder_path
        destination_folder = os.path.join(destination_folder, '')
        zip_filename = f"image{x}.zip"
        zip_filepath = os.path.join(destination_folder, zip_filename)

        with zipfile.ZipFile(zip_filepath, 'w') as myzip:
            for filename in os.listdir(source_folder):
                file_path = os.path.join(source_folder, filename)
                myzip.write(file_path, filename)

        print("Files have been successfully zipped for image{x} to", zip_filepath)

        source_file = zip_filepath
        destination_folder = '/content/drive/MyDrive'

        shutil.copy(source_file, destination_folder)
        print(f"Zip file for image{x} has been successfully copied to Google Drive folder.")

    except Exception as e:
        print(f"Error processing {url_file_path}: {e}")

if __name__ == '__main__':
    txt_files = list(range(1, 6))

    for file in txt_files:
        print(f"Processing image{file}:")
        process_txt(file)

流式输出内容被截断，只能显示最后 5000 行内容。
File 60b094d90fb343e87f36454517e554bd.jpg downloaded successfully.
File 02c1fbb062aa5000a885533ac7d2bc3d.jpg downloaded successfully.
File 3819075025fb5dc87022504118cde079.jpg downloaded successfully.
File OL3_02_000025.jpg downloaded successfully.
File 599cf257a518a3a49c8901a647f95e47.jpg downloaded successfully.
File OL3_02_000020.jpg downloaded successfully.
File 34049b1b5dfb51ea9e3d6f3afea18433.jpg downloaded successfully.
File 5ebc70370655949e23870c767c920ff6.jpg downloaded successfully.
File 042b9d44d7cb5545c2baa1cbdd2aa183.jpg downloaded successfully.
File bb4b4b5b30459e5eb12865f83bb46d7d.jpg downloaded successfully.
File IMG_2078.jpg downloaded successfully.
File 16b52b885f5ec2c5aceec2c876980a9a.jpg downloaded successfully.
File A7R03153.jpg downloaded successfully.
File eb6be01d512c589a49128b722b79bfbc.jpg downloaded successfully.File 2b3d318c15014030da4ed597bebad00a.jpg downloaded successfully.

File 20216f81a688b48266450a2b0b239d80.jpg downloaded s